necessary packages

In [1]:
import numpy as np
from ipywidgets import interact
import plotly.graph_objects as go
from scipy.special import gamma
from scipy.integrate import quad as integrate

define function for updating the layout of the figure:
* updates the layout of a plotly figure with predefined settings
* the **title** dict includes all title properties: **'x'** specifies the position of the title along the x-axis, **'xanchor'** specifies the anchor point of the title along the x-axis

In [2]:
def update_layout_of_graph(fig: go.Figure, title: str = 'Plot') -> go.Figure:
    fig.update_layout(
        width=1000,
        height=500,
        autosize=False,
        plot_bgcolor='black',
        title={'text': title, 'x': 0.5, 'xanchor': 'center'},  # dict includes all title properties; specifies the position of the title along the x-axis
        xaxis_title='input values',
        yaxis_title='output values',
        legend=dict(yanchor="top", y=0.5, xanchor="right", x=0.95),
        xaxis=dict(showline=True, linewidth=1, linecolor='black'),
        yaxis=dict(showline=True, linewidth=1, linecolor='black')
    )
    return fig

define function for creating a line plot:
* creates a Plotly scatter trace representing a line plot
* **x_lines** x-coordinates of the points on the line with an empty default array
* **y_lines** y-coordinates of the points on the line with an empty default array

In [3]:
def line_plot(x_lines: np.array = np.array([]), y_lines: np.array = np.array([]), title: str = 'predicted function',
show_legend: bool = True, visible: bool = True, ) -> go.Scatter:

    scatter = go.Scatter(x=x_lines, y=y_lines, line=dict(color="orange", width=3), name=title, showlegend= show_legend, visible=visible)
    return scatter

define function for a scatter plot:
* creates a scatter plot with Plotly
* **x_dots** is an array of x-coordinates for the data points in the scatter plot with an empty default array
* **y_dots** is an array of y-coordinates for the data points in the scatter plot with an empty default array

In [4]:
def scatter_plot(visible: bool = True, x_dots: np.array = np.array([]), y_dots: np.array = np.array([]), name_dots: str = 'observed points',
show_legend: bool = True, color: str = 'red') -> go.Scatter:

    scatter = go.Scatter(x=x_dots, y=y_dots, visible=visible, mode='markers', name=name_dots, marker=dict(color=color, size=8), showlegend=show_legend)
    return scatter

define function for plotting the uncertainty area:
* generates an area plot to represent the uncertainty using the Plotly
* **x_lines** an array of x-coordinates for the lines outlining the area plot with an empty default array
* **y_lower** an array of y-coordinates representing the lower boundary of the uncertainty area plot with an empty default array
* **y_upper** an array of y-coordinates representing the upper boundary of the uncertainty area plot with an empty default array

In [5]:
def uncertainty_area_plot(visible: bool = True, x_lines: np.array = np.array([]), y_lower: np.array = np.array([]), y_upper: np.array = np.array([]), name: str = 'mean plus/minus standard deviation') -> go.Scatter:

    scatter = go.Scatter(x=np.concatenate((x_lines, x_lines[::-1])), y=np.concatenate((y_upper, y_lower[::-1])), visible=visible, fill='toself', fillcolor='rgba(189, 195, 199, 0.5)', line=dict(color='rgba(200, 200, 200, 0)'), hoverinfo='skip', showlegend=True, name=name)
    return scatter

define squared exponential kernel:
* **variable1** & **variable2** correspond to t & t'
* **sigma** describes the average distance away from the function mean and l determines the reach of influence on neighbors
* **sigma** & **length** is assigned the float standard value 1
* **variable_1** & **variable_2** expect an array

In [6]:
class SquaredExponentialKernel:
    def __init__(self, sigma: float = 1, length: float = 1, alpha: float = 0.75, beta: float = 0.75):
        self.sigma = sigma
        self.length = length
        self.alpha = alpha
        self.beta = beta

    def __call__(self, tau_1: np.array, tau_2: np.array):
        def kernel(variable_1, variable_2):
            k_SE = float(self.sigma**2 * np.exp(-(np.linalg.norm(variable_1 - variable_2)**2) /(2 * self.length**2)))
            return k_SE
        return kernel(tau_1, tau_2)

visualize the squared exponential kernel:
* only for testing the squared exponential kernel and monitor the effect of changing the parameters
* **@interact** generates a user interface which allows for an interactive regulation of the parameters length, sigma and variable_2 in the defined ranges
* the function **update** is called whenever the values of the sliders are changed
* **batch_update()** ensures that all updates to the figure within the block are batched together

In [7]:
x_lines = np.arange(-10, 10, 0.1)
kernel = SquaredExponentialKernel()  # initialize an instance of the class SquaredExponentialKernel, length and sigma have already standard values
data = line_plot(x_lines=x_lines, y_lines=np.array([kernel(x, 0) for x in x_lines]))  # generates the data for the line distribution

fig = go.FigureWidget(data)  # generates an interactive plot-widget with the passed data
fig = update_layout_of_graph(fig, title='squared exponential kernel')

@interact(length=(0.1, 3, 0.1), sigma=(0.1, 3, 0.1), variable_2=(-10, 10, 0.1))
def update(length=1, sigma=1, variable_2=0):
    with fig.batch_update():
        kernel = SquaredExponentialKernel(sigma=sigma, length=length)  # a new instance of the class SquaredExponentialKernel with the updated length and sigma value is generated
        fig.data[0].y = np.array([kernel(x, variable_2) for x in x_lines])  # updates the y-values plot data, data can contain multiply scatter plots
        # with data[0].y the y-coordinates of the first scatter plot are accessed

fig

interactive(children=(FloatSlider(value=1.0, description='length', max=3.0, min=0.1), FloatSlider(value=1.0, d…

FigureWidget({
    'data': [{'line': {'color': 'orange', 'width': 3},
              'name': 'predicted function',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'ca5b7775-9abc-4c81-9d42-32cf72ff7298',
              'visible': True,
              'x': array([-1.00000000e+01, -9.90000000e+00, -9.80000000e+00, -9.70000000e+00,
                          -9.60000000e+00, -9.50000000e+00, -9.40000000e+00, -9.30000000e+00,
                          -9.20000000e+00, -9.10000000e+00, -9.00000000e+00, -8.90000000e+00,
                          -8.80000000e+00, -8.70000000e+00, -8.60000000e+00, -8.50000000e+00,
                          -8.40000000e+00, -8.30000000e+00, -8.20000000e+00, -8.10000000e+00,
                          -8.00000000e+00, -7.90000000e+00, -7.80000000e+00, -7.70000000e+00,
                          -7.60000000e+00, -7.50000000e+00, -7.40000000e+00, -7.30000000e+00,
                          -7.20000000e+00, -7.10000000e+00, -7.000000

functions for creating the covariance matrix, each of them were implemented separately in such a way as to make them comprehensible with the notes, which is why they are referenced:
* **val_1** & **val_2** are equal to a certain element of the arrays **t** & **t'**
* **alpha**, **R** & **C** are set to a specific value

In [8]:
def k_fu(val_1, val_2, cov_func, R=4, C=330e-6, L=1e-3):
    alpha = cov_func.alpha
    beta = cov_func.beta
    l = cov_func.length

    c = 1/(R*C) * cov_func(val_1, val_2)

    def integrand_a(tau):
        a = -1 / (gamma(1-alpha) * l**2)
        return a * (tau - val_2) / (val_1 - tau)**alpha * cov_func(tau, val_2)

    def integrand_b(tau):
        b = -1 / (gamma(1+beta) * l**2 * L * C)
        return b * (tau - val_2) * (val_1 - tau)**beta * cov_func(tau, val_2)

    integral_val_a, _ = integrate(integrand_a, 0, val_1, epsabs=1e-9, epsrel=1e-9, limit=10000)
    integral_val_b, _ = integrate(integrand_b, 0, val_1, epsabs=1e-9, epsrel=1e-9, limit=10000)
    return integral_val_a + 1 / (L * C)* integral_val_b + c

In [9]:
def k_uf(val_1, val_2, cov_func, R=4, C=330e-6, L=1e-3):
    alpha = cov_func.alpha
    beta = cov_func.beta
    l = cov_func.length

    c = 1/(R*C) * cov_func(val_1, val_2)

    def integrand_a(tau):
        a = 1 / (gamma(1-alpha) * l**2)
        return a * (val_1 - tau) / (val_2 - tau)**alpha * cov_func(val_1, tau)

    def integrand_b(tau):
        b = 1 / (gamma(1+beta) * l**2 * L * C)
        return b * (val_1 - tau) * (val_2 - tau)**beta * cov_func(val_1, tau)

    integral_val_a, _ = integrate(integrand_a, 0, val_2, epsabs=1e-9, epsrel=1e-9, limit=10000)
    integral_val_b, _ = integrate(integrand_b, 0, val_2, epsabs=1e-9, epsrel=1e-9, limit=10000)
    return integral_val_a + 1 / (L * C)* integral_val_b + c

In [10]:
def k_uu(val_1, val_2, cov_func):
    return cov_func(val_1, val_2)

In [11]:
def k_ff(val_1, val_2, cov_func, R=4, C=330e-6, L=1e-3):
    alpha = cov_func.alpha
    beta = cov_func.beta
    l = cov_func.length

    # simpler integrals, see notes:
    a = 1 / (R*C) * k_uf(val_1, val_2, cov_func, R, C, L)

    def integrand_c(tau):
        c_const = -1 / (gamma(1-alpha) * l**2 * R)
        return c_const * (tau - val_2) / (val_1 - tau)**alpha * cov_func(tau, val_2)

    def integrand_b(tau):
        b_const = -1 / (gamma(1+beta) * l**2 * R * C * L)
        return b_const * (tau - val_2) * (val_1 - tau)**beta * cov_func(tau, val_2)

    integral_val_c, _ = integrate(integrand_c, 0, val_1, epsabs=1e-8, epsrel=1e-8, limit=10000)
    integral_val_b, _ = integrate(integrand_b, 0, val_1, epsabs=1e-8, epsrel=1e-8, limit=10000)

    # more complex integral g:
    def g():
        def A(val_1, val_2, tau):
            a = C/gamma(1-alpha) * 1/(val_2 - tau)**alpha * 1/l**2 * (val_1 - tau)
            return a * cov_func(val_1, tau)

        def B(val_1, val_2):
            integrand = lambda tau: A(val_1, val_2, tau)
            integral_val, _ = integrate(integrand, 0, val_2, epsabs=1e-8, epsrel=1e-8, limit=10000)
            return integral_val

        def derivative_B(tau, val_2, h= 0.01*np.sqrt(2e-53)):
            def B_tau(tau):
                return B(tau, val_2)
            dB = (B_tau(tau + h) - B_tau(tau - h)) / (2*h)
            return dB

        def double_integral(val_1, val_2):
            integrand = lambda tau: 1/gamma(1 - alpha) * 1/(val_1 - tau)**alpha * derivative_B(tau, val_2)
            integral_val, _ = integrate(integrand, 0, val_1, epsabs=1e-8, epsrel=1e-8, limit=10000)
            return integral_val

        g = double_integral(val_1, val_2)
        return g

    # more complex integral f:
    def f():
        def A(val_1, val_2, tau):
            a = 1/(L*gamma(1+beta)) * (val_2 - tau)**beta * 1/l**2 * (val_1 - tau)
            return a * cov_func(val_1, tau)

        def B(val_1, val_2):
            integrand = lambda tau: A(val_1, val_2, tau)
            integral_val, _ = integrate(integrand, 0, val_2, epsabs=1e-8, epsrel=1e-8, limit=10000)
            return integral_val

        def derivative_B(tau, val_2, h= 0.01*np.sqrt(2e-53)):
            def B_tau(tau):
                return B(tau, val_2)
            dB = (B_tau(tau + h) - B_tau(tau - h)) / (2*h)
            return dB

        def double_integral(val_1, val_2):
            integrand = lambda tau: 1/gamma(1 - alpha) * 1/(val_1 - tau)**alpha * derivative_B(tau, val_2)
            integral_val, _ = integrate(integrand, 0, val_1, epsabs=1e-8, epsrel=1e-8, limit=10000)
            return integral_val

        f = double_integral(val_1, val_2)
        return f

    # more complex integral e:
    def e():
        def A(val_1, val_2, tau):
            a = C/gamma(1-alpha) * 1/(val_2 - tau)**alpha * 1/l**2 * (val_1 - tau)
            return a * cov_func(val_1, tau)

        def B(val_1, val_2):
            integrand = lambda tau: A(val_1, val_2, tau)
            integral_val, _ = integrate(integrand, 0, val_2, epsabs=1e-8, epsrel=1e-8, limit=10000)
            return integral_val

        def derivative_B(tau, val_2, h= 0.01*np.sqrt(2e-53)):
            def B_tau(tau):
                return B(tau, val_2)
            dB = (B_tau(tau + h) - B_tau(tau - h)) / (2*h)
            return dB

        def double_integral(val_1, val_2):
            integrand = lambda tau: 1/gamma(1 + beta) * (val_1 - tau)**beta * derivative_B(tau, val_2)
            integral_val, _ = integrate(integrand, 0, val_1, epsabs=1e-8, epsrel=1e-8, limit=10000)
            return integral_val

        e = double_integral(val_1, val_2)
        return e

    # more complex integral d:
    def d():
        def A(val_1, val_2, tau):
            a = 1/(L*gamma(1+beta)) * (val_2 - tau)**beta * 1/l**2 * (val_1 - tau)
            return a * cov_func(val_1, tau)

        def B(val_1, val_2):
            integrand = lambda tau: A(val_1, val_2, tau)
            integral_val, _ = integrate(integrand, 0, val_2, epsabs=1e-8, epsrel=1e-8, limit=10000)
            return integral_val

        def derivative_B(tau, val_2, h= 0.01*np.sqrt(2e-53)):
            def B_tau(tau):
                return B(tau, val_2)
            dB = (B_tau(tau + h) - B_tau(tau - h)) / (2*h)
            return dB

        def double_integral(val_1, val_2):
            integrand = lambda tau: 1/gamma(1 + beta) * (val_1 - tau)**beta * derivative_B(tau, val_2)
            integral_val, _ = integrate(integrand, 0, val_1, epsabs=1e-8, epsrel=1e-8, limit=10000)
            return integral_val

        d = double_integral(val_1, val_2)
        return d

    return a + integral_val_b + integral_val_c + g() + f() + e() + d()

testing the implemented functions above:

In [12]:
val_1 = 0.1
val_2 = 0.1
x = np.arange(0.01, 3, 0.01)
cov_func = SquaredExponentialKernel()

for i in range(len(x)):
    kuf = k_uf(x[i], x[i], cov_func)
    kfu = k_fu(x[i], x[i], cov_func)
    kuu = k_uu(x[i], x[i], cov_func)
    kff = k_ff(x[i], x[i], cov_func)
    print('kuf = ', kuf, 'kuu = ', kuu, 'kfu = ', kfu, 'kff = ', kff)

/var/folders/db/wdxh9ynd7_bfr2nwqswb2nn80000gn/T/ipykernel_55963/72069979.py:28: IntegrationWarning:

The integral is probably divergent, or slowly convergent.

/var/folders/db/wdxh9ynd7_bfr2nwqswb2nn80000gn/T/ipykernel_55963/72069979.py:78: IntegrationWarning:

The integral is probably divergent, or slowly convergent.



kuf =  11489746.107366322 kuu =  1.0 kfu =  11489746.107366322 kff =  8704353112.589169
kuf =  77282440.25091717 kuu =  1.0 kfu =  77282440.25091717 kff =  58547303226.76795
kuf =  235649255.63191846 kuu =  1.0 kfu =  235649255.63191846 kff =  178522163376.95572
kuf =  519707327.23503035 kuu =  1.0 kfu =  519707327.23503035 kff =  393717672190.6271
kuf =  959727534.2283741 kuu =  1.0 kfu =  959727534.2283741 kff =  727066313888.5532
kuf =  1584010237.2723722 kuu =  1.0 kfu =  1584010237.2723722 kff =  1200007755640.0554
kuf =  2419347182.9609118 kuu =  1.0 kfu =  2419347182.9609118 kff =  1832838775169.9858
kuf =  3491302657.132474 kuu =  1.0 kfu =  3491302657.132474 kff =  2644926255691.4243
kuf =  4824399765.774778 kuu =  1.0 kfu =  4824399765.774778 kff =  3654848307803.151
kuf =  6442251013.744954 kuu =  1.0 kfu =  6442251013.744954 kff =  4880493192762.515
kuf =  8367653570.0851755 kuu =  1.0 kfu =  8367653570.0851755 kff =  6339131493179.1045
kuf =  10622660852.768368 kuu =  1.0 

a function for creating the sub and covariance matrix as described in the notes:
* works only correctly for squared matrices [n1 x n2] where n1 = n2

In [13]:
def create_sub_matrix(variable_1, variable_2, specified_cov_function, cov_function):
    n1 = len(variable_1)
    n2 = len(variable_2)
    sub_matrix = np.zeros((n1, n2))

    for i, val_1 in enumerate(variable_1):
        for j, val_2 in enumerate(variable_2):
            sub_matrix[i, j] = specified_cov_function(val_1, val_2, cov_function)

    return sub_matrix

a function for creating the covariance matrix as described in the notes:
* works only correctly for squared matrices [2*n1 x 2*n2] where n1 = n2

In [14]:
def create_cov_matrix(variable_1, variable_2, cov_function):
    n1 = len(variable_1)
    n2 = len(variable_2)
    cov = np.zeros((2*n1, 2*n2))

    kuu_sub_matrix = create_sub_matrix(variable_1, variable_2, k_uu, cov_function)
    kuf_sub_matrix = create_sub_matrix(variable_1, variable_2, k_uf, cov_function)
    kfu_sub_matrix = create_sub_matrix(variable_1, variable_2, k_fu, cov_function)
    kff_sub_matrix = create_sub_matrix(variable_1, variable_2, k_ff, cov_function)

    for i in range(2*n1):
        for j in range(2*n2):

            if i <= n1 - 1 and j <= n2 - 1:
                #print('kuu')
                cov[i, j] = kuu_sub_matrix[i, j]
            elif i <= n1 - 1 and n2 <= j <= 2*n2 - 1:
                #print('kuf')
                cov[i, j] = kuf_sub_matrix[i, j - n2]
            elif n1 <= i <= 2*n1 - 1 and j <= n2 - 1:
                #print('kfu')
                cov[i, j] = kfu_sub_matrix[i - n1, j]
            else:
                #print('kff')
                cov[i, j] = kff_sub_matrix[i - n1, j - n2]

    return cov

testing the implemented functions for creating the covariance matrix above

In [15]:
variable_1 = [1, 1, 1, 1]
variable_2 = [1, 1, 1, 1]
#variable_1 = np.array([0.3, 0.8, 1.4, 1.8, 2])
#variable_2 = np.array([0.3, 0.8, 1.4, 1.8, 2])

cov_func = SquaredExponentialKernel()
cov = create_cov_matrix(variable_1, variable_2, cov_func)

print('covariance matrix = ', cov)

/var/folders/db/wdxh9ynd7_bfr2nwqswb2nn80000gn/T/ipykernel_55963/72069979.py:28: IntegrationWarning:

The integral is probably divergent, or slowly convergent.

/var/folders/db/wdxh9ynd7_bfr2nwqswb2nn80000gn/T/ipykernel_55963/72069979.py:78: IntegrationWarning:

The integral is probably divergent, or slowly convergent.



covariance matrix =  [[1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  2.74498160e+12 2.74498160e+12 2.74498160e+12 2.74498160e+12]
 [1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  2.74498160e+12 2.74498160e+12 2.74498160e+12 2.74498160e+12]
 [1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  2.74498160e+12 2.74498160e+12 2.74498160e+12 2.74498160e+12]
 [1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  2.74498160e+12 2.74498160e+12 2.74498160e+12 2.74498160e+12]
 [2.74498160e+12 2.74498160e+12 2.74498160e+12 2.74498160e+12
  2.07953151e+15 2.07953151e+15 2.07953151e+15 2.07953151e+15]
 [2.74498160e+12 2.74498160e+12 2.74498160e+12 2.74498160e+12
  2.07953151e+15 2.07953151e+15 2.07953151e+15 2.07953151e+15]
 [2.74498160e+12 2.74498160e+12 2.74498160e+12 2.74498160e+12
  2.07953151e+15 2.07953151e+15 2.07953151e+15 2.07953151e+15]
 [2.74498160e+12 2.74498160e+12 2.74498160e+12 2.74498160e+12
  2.07953151e+15 2.07953151e+15 2.07953151

define function for calculating q_u transposed


In [16]:
def q_u_transposed(variable_1, variable_2, cov_function):
    n1 = len(variable_1)
    n2 = len(variable_2)
    q_u_trans = np.zeros((n1, 2*n2))

    kuu_sub_matrix = create_sub_matrix(variable_1, variable_2, k_uu, cov_function)
    kuf_sub_matrix = create_sub_matrix(variable_1, variable_2, k_uf, cov_function)

    for i in range(n1):
        for j in range(2*n2):

            if j <= n2 - 1:
                q_u_trans[i, j] = kuu_sub_matrix[i, j]
            else:
                q_u_trans[i, j] = kuf_sub_matrix[i, j - n2]

    return q_u_trans

In [17]:
def q_f_transposed(variable_1, variable_2, cov_function):
    n1 = len(variable_1)
    n2 = len(variable_2)
    q_f_trans = np.zeros((n1, 2*n2))

    kfu_sub_matrix = create_sub_matrix(variable_1, variable_2, k_fu, cov_function)
    kff_sub_matrix = create_sub_matrix(variable_1, variable_2, k_ff, cov_function)

    for i in range(n1):
        for j in range(2*n2):

            if j <= n2 - 1:
                q_f_trans[i, j] = kfu_sub_matrix[i, j]
            else:
                q_f_trans[i, j] = kff_sub_matrix[i, j - n2]

    return q_f_trans

testing if the function q_u_transposed works properly

In [18]:
variable_1 = [1, 1, 1, 1]
variable_2 = [1, 2, 1, 1]
cov_func = SquaredExponentialKernel()

q_u_trans = q_u_transposed(variable_1, variable_2, cov_func)
print('q_u transposed = ', q_u_trans)

q_u transposed =  [[1.00000000e+00 6.06530660e-01 1.00000000e+00 1.00000000e+00
  2.74498160e+12 3.90005833e+12 2.74498160e+12 2.74498160e+12]
 [1.00000000e+00 6.06530660e-01 1.00000000e+00 1.00000000e+00
  2.74498160e+12 3.90005833e+12 2.74498160e+12 2.74498160e+12]
 [1.00000000e+00 6.06530660e-01 1.00000000e+00 1.00000000e+00
  2.74498160e+12 3.90005833e+12 2.74498160e+12 2.74498160e+12]
 [1.00000000e+00 6.06530660e-01 1.00000000e+00 1.00000000e+00
  2.74498160e+12 3.90005833e+12 2.74498160e+12 2.74498160e+12]]


training: page 113 in GP for ML

In [19]:
def log_marginal_likelihood(data_x, data_y, cov_func, noise):
    n = len(data_x)
    y = np.concatenate([data_y, data_y])
    y_transposed = y.T
    K = create_cov_matrix(data_x, data_x, cov_func) + noise * np.identity(2*n)
    K_inverse = np.linalg.inv(K)

    a = n/2 * np.log(2*np.pi)  # normalization constant
    b = 1/2 * np.log(np.linalg.norm(K))  # complexity penalty
    c = 1/2 * np.dot(y, np.dot(K_inverse, y_transposed))  # involving observed targets
    log_ml = c + b + a  # is actually the negative log marginal likelihood
    return log_ml

sigma_vals = np.arange(0.1, 2, 0.05)
length_vals = np.arange(0.1, 2, 0.05)
memory = {'sigma': 1, 'length': 1, 'negative_log_marginal_likelihood': 1000}

data_x = np.array([0.08919118309982221, 0.48573526800672523, 1.067487500149583, 2.01754330372101, 2.903425000149563, 4.768659375149524, 6.594831250149485, 8.460065625149445, 9.504987500149424])
data_y = np.array([0.6626339, -0.6613839, -1.459256, -0.1826344, 0.7235726, -0.358959, 0.1779366, -0.08830133, -0.001224956])

for sigma_index, sigma_val in enumerate(sigma_vals):
    for l_index, length_val in enumerate(length_vals):
        cov_func = SquaredExponentialKernel(sigma=sigma_val, length=length_val)
        negative_log_marginal_likelihood = log_marginal_likelihood(data_x, data_y, cov_func, noise = 0.01)
        print('likelihood = ', negative_log_marginal_likelihood)

        if negative_log_marginal_likelihood < memory['negative_log_marginal_likelihood']:
            memory['length'] = length_val
            memory['sigma'] = sigma_val
            memory['negative_log_marginal_likelihood'] = negative_log_marginal_likelihood

        print('obtained sigma and l values that minimize the negative log marginal likelihood: ', 'sigma = ', memory['sigma'], 'length = ', memory['length'])

/var/folders/db/wdxh9ynd7_bfr2nwqswb2nn80000gn/T/ipykernel_55963/72069979.py:28: IntegrationWarning:

The integral is probably divergent, or slowly convergent.

/var/folders/db/wdxh9ynd7_bfr2nwqswb2nn80000gn/T/ipykernel_55963/72069979.py:78: IntegrationWarning:

The integral is probably divergent, or slowly convergent.



KeyboardInterrupt: 

testing the function above

In [20]:
cov_func = SquaredExponentialKernel(sigma=0.2, length=1.9)  # (0.53, 1.4);
negative_log_marginal_likelihood = log_marginal_likelihood(data_x, data_y, cov_func, noise = 0.01)
print(negative_log_marginal_likelihood)

/var/folders/db/wdxh9ynd7_bfr2nwqswb2nn80000gn/T/ipykernel_55963/72069979.py:28: IntegrationWarning:

The integral is probably divergent, or slowly convergent.

/var/folders/db/wdxh9ynd7_bfr2nwqswb2nn80000gn/T/ipykernel_55963/72069979.py:78: IntegrationWarning:

The integral is probably divergent, or slowly convergent.

/var/folders/db/wdxh9ynd7_bfr2nwqswb2nn80000gn/T/ipykernel_55963/72069979.py:17: IntegrationWarning:

The integral is probably divergent, or slowly convergent.



26.217021271605653


define class GPR:
* **data_x** is the input of the observed points
* **data_y** is the output of the observed points
* **noise** is the noise in the observed data
* **memory** is a variable where the mean, the covariance matrix and the variance is stored
* **3e-7** add this value to the noise in order to ensure stability when inverting the covariance matrix

In [21]:
class GPR:
    def __init__(self, data_x, data_y, cov_func= SquaredExponentialKernel(sigma=memory['sigma'], length=memory['length']), noise=0.01):
        print('sigma memory = ', memory['sigma'])
        print('length memory = ', memory['length'])
        self.data_x = data_x
        self.data_y = np.concatenate([data_y, data_y])
        self.cov_func = cov_func
        self.noise = noise
        self.memory = None

        n = len(data_x)
        self.inv_cov_matrix_of_input_data = np.linalg.inv(create_cov_matrix(data_x, data_x, cov_func) + (noise + 3e-7)**2 * np.identity(2*n))

    def predict(self, at_values):
        q_u_trans = q_u_transposed(at_values, self.data_x, self.cov_func)
        q_f_trans = q_f_transposed(at_values, self.data_x, self.cov_func)

        # methode paper:
        mean_at_values_u = np.dot(q_u_trans, np.dot(self.data_y, self.inv_cov_matrix_of_input_data.T).T).flatten()
        #mean_at_values_f = np.dot(q_f_trans, np.dot(self.data_y, self.inv_cov_matrix_of_input_data.T).T).flatten()

        covariance_matrix_u = create_sub_matrix(at_values, at_values, k_uu, self.cov_func) - np.dot(q_u_trans, np.dot(self.inv_cov_matrix_of_input_data, q_u_trans.T))
        #covariance_matrix_f = create_sub_matrix(at_values, at_values, k_ff, self.cov_func) - np.dot(q_f_trans, np.dot(self.inv_cov_matrix_of_input_data, q_f_trans.T))

        variance_u = np.abs(np.diag(covariance_matrix_u))
        #variance_f = np.abs(np.diag(covariance_matrix_f))

        self.memory = {'mean': mean_at_values_u, 'covariance_matrix':covariance_matrix_u, 'variance': variance_u}

        return mean_at_values_u

defining function for plotting the GPR:
* **data_x** is a list of observed input data points
* **data_y** is a list of observed output data points
* **x** is equivalent to **at_values** in the GPR, the values we want to do predictions on

In [22]:
def plot_GPR(data_x, data_y, model, x, data_x_mv, data_y_mv, visible=True) -> list:
    mean = model.predict(x)
    standard_deviation = np.sqrt(model.memory['variance'])
    data = []

    for i in range(1, 4):
        data.append(uncertainty_area_plot(x_lines=x, y_lower=mean -i * standard_deviation, y_upper=mean + i*standard_deviation, name=f'mean plus/minus {i}*standard deviation', visible=visible))

    data.append(scatter_plot(x_dots=data_x_mv, y_dots=data_y_mv, visible=visible, color='blue'))
    data.append(scatter_plot(x_dots=data_x, y_dots=data_y, visible=visible, color='red'))
    data.append(line_plot(x_lines=x, y_lines=mean, visible=visible))
    return data

initializing the training data

In [23]:
x_values = np.array([0.08919118309982221, 0.48573526800672523, 1.067487500149583, 2.01754330372101, 2.903425000149563, 4.768659375149524, 6.594831250149485, 8.460065625149445, 9.504987500149424])
y_values = np.array([0.6626339, -0.6613839, -1.459256, -0.1826344, 0.7235726, -0.358959, 0.1779366, -0.08830133, -0.001224956])

x_values_mv = np.array([0.0, 0.029730524574942258, 0.059460853837382234, 0.08919118309982221, 0.1189215123622622, 0.14865184162470219, 0.1783821708871422, 0.2081125001495821, 0.2374093751495821, 0.2569406251495821, 0.2667062501495821, 0.2764718751495821, 0.28623750014958205, 0.3057687501495821, 0.3350656251495822, 0.3727330358638679, 0.41040044657815367, 0.4480678572924394, 0.48573526800672523, 0.5234026787210109, 0.5610700894352967, 0.5987375001495824, 0.6391950894352967, 0.679652678721011, 0.7201102680067254, 0.7605678572924397, 0.801025446578154, 0.8414830358638684, 0.8819406251495826, 0.9056571430067255, 0.9293736608638683, 0.9530901787210113, 0.9768066965781541, 1.000523214435297, 1.02423973229244, 1.047956250149583, 1.057721875149583, 1.067487500149583, 1.077253125149583, 1.096784375149583, 1.126081250149583, 1.1511928572924401, 1.1763044644352971, 1.201416071578154, 1.226527678721012, 1.251639285863869, 1.276750893006726, 1.301862500149583, 1.321393750149583, 1.331159375149583, 1.350690625149583, 1.379987500149583, 1.416259821578154, 1.452532143006726, 1.488804464435298, 1.525076785863869, 1.56134910729244, 1.597621428721012, 1.6338937501495832, 1.674351339435298, 1.714808928721012, 1.755266518006726, 1.795724107292441, 1.836181696578155, 1.876639285863869, 1.917096875149584, 1.9422084822924401, 1.9673200894352971, 1.9924316965781539, 2.01754330372101, 2.042654910863867, 2.067766518006724, 2.092878125149581, 2.11240937514958, 2.12217500014958, 2.13194062514958, 2.151471875149579, 2.1807687501495794, 2.218436160863864, 2.2561035715781492, 2.293770982292434, 2.331438393006719, 2.3691058037210033, 2.406773214435288, 2.444440625149573, 2.484898214435287, 2.5253558037209998, 2.565813393006714, 2.606270982292427, 2.64672857157814, 2.6871861608638543, 2.727643750149567, 2.751360268006709, 2.775076785863852, 2.798793303720994, 2.8225098215781372, 2.8462263394352787, 2.869942857292421, 2.893659375149564, 2.903425000149563, 2.913190625149563, 2.922956250149563, 2.942487500149563, 2.971784375149562, 2.9968959822924193, 3.022007589435275, 3.047119196578132, 3.0722308037209882, 3.097342410863845, 3.1224540180067017, 3.147565625149558, 3.167096875149558, 3.176862500149558, 3.1866281251495567, 3.2061593751495567, 3.235456250149556, 3.273123660863841, 3.3107910715781257, 3.3484584822924113, 3.386125893006696, 3.423793303720981, 3.4614607144352663, 3.499128125149551, 3.539585714435264, 3.580043303720977, 3.620500893006691, 3.660958482292405, 3.7014160715781177, 3.7418736608638308, 3.782331250149545, 3.8060477680066866, 3.829764285863829, 3.853480803720972, 3.877197321578114, 3.9009138394352574, 3.924630357292399, 3.9483468751495425, 3.958112500149541, 3.967878125149541, 3.9776437501495407, 3.9971750001495394, 4.026471875149539, 4.064139285863824, 4.10180669657811, 4.139474107292394, 4.17714151800668, 4.214808928720965, 4.252476339435249, 4.290143750149534, 4.330601339435248, 4.371058928720961, 4.411516518006675, 4.451974107292387, 4.492431696578101, 4.532889285863815, 4.573346875149528, 4.597063393006671, 4.6207799108638135, 4.644496428720955, 4.668212946578098, 4.6919294644352405, 4.715645982292382, 4.739362500149524, 4.758893750149524, 4.768659375149524, 4.778425000149523, 4.797956250149523, 4.8272531251495225, 4.852364732292379, 4.877476339435236, 4.902587946578093, 4.927699553720949, 4.952811160863805, 4.977922768006663, 5.003034375149519, 5.012800000149519, 5.022565625149519, 5.032331250149518, 5.051862500149518, 5.081159375149517, 5.118826785863802, 5.1564941965780875, 5.194161607292372, 5.231829018006657, 5.269496428720942, 5.307163839435227, 5.344831250149512, 5.385288839435226, 5.425746428720938, 5.4662040180066525, 5.506661607292365, 5.547119196578079, 5.587576785863793, 5.628034375149506, 5.651750893006648, 5.67546741086379, 5.699183928720933, 5.722900446578075, 5.746616964435217, 5.77033348229236, 5.794050000149502, 5.813581250149502, 5.823346875149502, 5.833112500149501, 5.852643750149501, 5.8819406251495, 5.919608035863785, 5.957275446578071, 5.994942857292354, 6.03261026800664, 6.070277678720925, 6.10794508943521, 6.145612500149495, 6.186070089435208, 6.226527678720921, 6.266985268006636, 6.307442857292348, 6.347900446578062, 6.388358035863776, 6.428815625149489, 6.452532143006631, 6.476248660863773, 6.499965178720916, 6.523681696578058, 6.5473982144352005, 6.571114732292344, 6.594831250149485, 6.604596875149485, 6.614362500149484, 6.6241281251494835, 6.643659375149484, 6.672956250149483, 6.69806785729234, 6.723179464435197, 6.748291071578054, 6.77340267872091, 6.798514285863766, 6.823625893006623, 6.84873750014948, 6.868268750149479, 6.878034375149479, 6.897565625149479, 6.926862500149478, 6.963134821578048, 6.99940714300662, 7.035679464435191, 7.07195178586376, 7.108224107292331, 7.144496428720902, 7.180768750149473, 7.221226339435186, 7.261683928720899, 7.302141518006613, 7.342599107292326, 7.3830566965780395, 7.423514285863754, 7.463971875149467, 7.489083482292323, 7.51419508943518, 7.539306696578038, 7.564418303720893, 7.5895299108637495, 7.614641518006605, 7.639753125149463, 7.659284375149462, 7.669050000149462, 7.678815625149462, 7.698346875149461, 7.727643750149461, 7.765311160863746, 7.802978571578031, 7.840645982292315, 7.878313393006601, 7.9159808037208865, 7.95364821443517, 7.991315625149455, 8.031773214435168, 8.072230803720883, 8.112688393006596, 8.153145982292308, 8.193603571578022, 8.234061160863737, 8.274518750149449, 8.298235268006591, 8.321951785863735, 8.345668303720878, 8.369384821578018, 8.39310133943516, 8.416817857292303, 8.440534375149445, 8.450300000149445, 8.460065625149445, 8.469831250149445, 8.489362500149445, 8.518659375149443, 8.543770982292301, 8.568882589435157, 8.593994196578013, 8.61910580372087, 8.644217410863728, 8.669329018006584, 8.69444062514944, 8.71397187514944, 8.72373750014944, 8.73350312514944, 8.75303437514944, 8.782331250149438, 8.819998660863725, 8.857666071578008, 8.895333482292292, 8.933000893006579, 8.970668303720863, 9.008335714435148, 9.046003125149433, 9.086460714435146, 9.12691830372086, 9.167375893006573, 9.207833482292287, 9.248291071577999, 9.288748660863714, 9.329206250149428, 9.35292276800657, 9.376639285863712, 9.400355803720855, 9.424072321577995, 9.447788839435137, 9.47150535729228, 9.495221875149424, 9.504987500149424, 9.514753125149422, 9.524518750149422, 9.544050000149422, 9.573346875149422, 9.608224107292278, 9.643101339435134, 9.67797857157799, 9.712855803720847, 9.747733035863703, 9.782610268006561, 9.817487500149417, 9.827253125149417, 9.849574553720846, 9.871895982292273, 9.894217410863702, 9.916538839435129, 9.938860268006557, 9.961181696577984, 9.983503125149413, 9.993268750149413, 10.0])  # x data for model validation

y_values_mv = np.array([1.0, 0.8866713, 0.7742162, 0.6626339, 0.5519243, 0.4420876, 0.3331237, 0.2250326, 0.1209512, 0.05264785, 0.01884616, -0.0147115, -0.04801726, -0.1138423, -0.2105016, -0.3315973, -0.4471096, -0.5570385, -0.6613839, -0.7601459, -0.8533246, -0.9409198, -1.027311, -1.105822, -1.176455, -1.239209, -1.294083, -1.341078, -1.380195, -1.399297, -1.415757, -1.429574, -1.440748, -1.44928, -1.455169, -1.458416, -1.459049, -1.459256, -1.459042, -1.457361, -1.451759, -1.443896, -1.433624, -1.420945, -1.405858, -1.388362, -1.368459, -1.346148, -1.327466, -1.317654, -1.297114, -1.264092, -1.218994, -1.171031, -1.120205, -1.066515, -1.009961, -0.9505431, -0.8882614, -0.8164804, -0.7438536, -0.6703811, -0.5960629, -0.5208989, -0.4448891, -0.3680336, -0.3209685, -0.2743802, -0.2282688, -0.1826344, -0.1374768, -0.09279619, -0.04859247, -0.0148689, 0.001815278, 0.0183758, 0.05111053, 0.09919094, 0.1594496, 0.2169515, 0.2716964, 0.3236845, 0.3729158, 0.4193902, 0.4631077, 0.5062569, 0.545501, 0.5808402, 0.6122743, 0.6398034, 0.6634275, 0.6831466, 0.6928046, 0.7011501, 0.708183, 0.7139033, 0.7183111, 0.7214063, 0.723189, 0.7235726, 0.7237449, 0.7237076, 0.7230101, 0.7204317, 0.7166994, 0.711768, 0.7056375, 0.6983081, 0.6897795, 0.680052, 0.6691253, 0.6599635, 0.6551479, 0.650179, 0.6397924, 0.6231288, 0.5994893, 0.5743678, 0.5477643, 0.5196787, 0.4901112, 0.4590616, 0.42653, 0.3905371, 0.3542035, 0.3175295, 0.2805148, 0.2431596, 0.2054638, 0.1674275, 0.145468, 0.1237353, 0.1022293, 0.08095015, 0.05989773, 0.03907209, 0.0184732, 0.0101197, 0.001824927, -0.006409152, -0.02268758, -0.04660301, -0.07658743, -0.1052107, -0.1324729, -0.1583739, -0.1829138, -0.2060925, -0.2279101, -0.2494602, -0.269075, -0.2867545, -0.3024987, -0.3163075, -0.328181, -0.3381191, -0.3430005, -0.3472299, -0.3508074, -0.353733, -0.3560067, -0.3576284, -0.3585983, -0.3589432, -0.358959, -0.3588712, -0.3583884, -0.3569086, -0.3548889, -0.3522794, -0.3490799, -0.3452906, -0.3409113, -0.3359421, -0.330383, -0.3280976, -0.3257344, -0.323295, -0.3181921, -0.309997, -0.2983563, -0.2859737, -0.2728491, -0.2589825, -0.244374, -0.2290234, -0.2129308, -0.1951162, -0.177125, -0.1589572, -0.1406127, -0.1220916, -0.1033939, -0.08451962, -0.07361971, -0.06283008, -0.05215075, -0.04158171, -0.03112296, -0.0207745, -0.01053633, -0.002260064, 0.001833884, 0.00589709, 0.01392752, 0.02571955, 0.04049254, 0.05458423, 0.06799462, 0.08072372, 0.09277151, 0.104138, 0.1148232, 0.1253613, 0.134938, 0.1435534, 0.1512075, 0.1579003, 0.1636318, 0.168402, 0.1707312, 0.1727381, 0.1744226, 0.1757847, 0.1768244, 0.1775417, 0.1779366, 0.1780132, 0.1780381, 0.1780114, 0.1778054, 0.1771205, 0.1761599, 0.1749056, 0.1733575, 0.1715157, 0.1693801, 0.1669508, 0.1642277, 0.1619476, 0.1607501, 0.1582434, 0.1542136, 0.1487101, 0.1428573, 0.1366552, 0.1301038, 0.123203, 0.115953, 0.1083536, 0.09959534, 0.09073398, 0.08176954, 0.07270202, 0.06353143, 0.05425777, 0.04488103, 0.0391389, 0.03345499, 0.0278293, 0.02226184, 0.0167526, 0.01130157, 0.005908776, 0.001794567, -0.000240862, -0.0022612, -0.006254728, -0.01212034, -0.01947156, -0.02648639, -0.03316482, -0.03950686, -0.0455125, -0.05118174, -0.05651458, -0.06177798, -0.06656493, -0.07087543, -0.07470949, -0.0780671, -0.08094827, -0.083353, -0.08453068, -0.08554823, -0.08640565, -0.08710294, -0.08764009, -0.08801711, -0.088234, -0.08828055, -0.08830133, -0.08829653, -0.08821096, -0.08789567, -0.08743971, -0.08683747, -0.08608896, -0.08519418, -0.08415312, -0.08296579, -0.08163218, -0.08051401, -0.0799263, -0.07931989, -0.07805231, -0.07601874, -0.07313399, -0.07006848, -0.06682219, -0.06339514, -0.05978732, -0.05599874, -0.05202939, -0.04763776, -0.04320465, -0.03873003, -0.03421393, -0.02965632, -0.02505722, -0.02041663, -0.0177375, -0.01508606, -0.01246229, -0.009866213, -0.007297815, -0.0047571, -0.002244066, -0.001224956, -0.0002130131, 0.0007915224, 0.002777438, 0.005695024, 0.00908409, 0.01233262, 0.01544063, 0.0184081, 0.02123504, 0.02392145, 0.02646732, 0.02714644, 0.02865242, 0.03008809, 0.03145346, 0.03274853, 0.03397329, 0.03512774, 0.03621189, 0.03666376, 0.0369688])  # y data for model validation

print(len(x_values_mv), len(y_values_mv))

x = np.arange(0.01, 10, 0.02)

350 350


plot the output of the GPR

In [24]:
model = GPR(x_values, y_values)
data = plot_GPR(data_x=x_values, data_y=y_values, x=x, model=model, data_x_mv=x_values_mv, data_y_mv=y_values_mv)
fig4 = go.Figure(data)
fig4 = update_layout_of_graph(fig=fig4, title=f'GPR with length {model.cov_func.length}, sigma {model.cov_func.sigma}, alpha {cov_func.alpha}, beta {model.cov_func.beta} and noise {model.noise}')
fig4.show()
fig4.write_html('/Users/simonjamnik/PycharmProjects/GaussianProcess/RLC_Schwingfall_voltage_0.75.html')

sigma memory =  1
length memory =  1


/var/folders/db/wdxh9ynd7_bfr2nwqswb2nn80000gn/T/ipykernel_55963/72069979.py:28: IntegrationWarning:

The integral is probably divergent, or slowly convergent.

/var/folders/db/wdxh9ynd7_bfr2nwqswb2nn80000gn/T/ipykernel_55963/72069979.py:78: IntegrationWarning:

The integral is probably divergent, or slowly convergent.



KeyboardInterrupt: 